# Submit job on Nersc

[**Common slurm commands**](https://github.com/NERSC/slurm-magic), such as **_%sbatch, %squeue, %scancel, %salloc, %srun, %scontrol_**

Any command that works at the command-line can be used in Jupyter Notebook by prefixing it with the **! ** character. For example, the **ls, pwd, and echo** commands can be run as follows: **!ls, !pwd, !echo** ([link](https://jakevdp.github.io/PythonDataScienceHandbook/01.05-ipython-and-shell-commands.html))

Besides **%cd**, other available shell-like magic functions are **%cat, %cp, %env, %ls, %man, %mkdir, %more, %mv, %pwd, %rm, and %rmdir**, any of which can be used without the % sign if `automagic` is on.

In [23]:
import subprocess

## execute batch job

In [24]:
subprocess.call(['ls', '-ltr'])

0

In [20]:
pwd

'/global/cscratch1/sd/pshuai/John_case'

In [1]:
cd /global/cscratch1/sd/pshuai/John_case/

/global/cscratch1/sd/pshuai


In [6]:
ls -ltr

total 151088
-rw-r--r-- 1 pshuai m1800 3138403 Dec 18 22:56 300A_H_Initial.h5
-rw-r----- 1 pshuai m1800     350 Dec 18 23:05 run.debug.sh~
-rw-rw---- 1 pshuai m1800   27621 Dec 19 15:01 '#slurm-9071284.out#'
-rw-r----- 1 pshuai m1800     351 Dec 20 10:47 run.debug.sh
-rw-rw---- 1 pshuai m1800 2222744 Dec 20 11:20 '#slurm-9080201.out#'
-rw-r----- 1 pshuai m1800     373 Dec 20 11:36 '#run.debug.sh#'
-rw-rw---- 1 pshuai m1800 5293630 Dec 20 12:18 slurm-9080696.out
-rw-rw---- 1 pshuai m1800 1157234 Dec 20 15:04 slurm-9081840.out
-rw-rw---- 1 pshuai m1800 2116248 Dec 20 22:18 slurm-9085903.out
-rw-rw---- 1 pshuai m1800 2116248 Dec 20 22:28 '#slurm-9085903.out#'
-rw-r--r-- 1 pshuai m1800 2597715 Dec 22 13:47 DatumH_River_filtered_6h_317.txt
-rw-r--r-- 1 pshuai m1800 2491546 Dec 22 13:47 DatumH_River_filtered_6h_316.txt
-rw-r--r-- 1 pshuai m1800 2596698 Dec 22 13:47 DatumH_River_filtered_6h_315.txt
-rw-r--r-- 1 pshuai m1800 2596914 Dec 22 13:47 DatumH_River_filtered_6h_314.txt
-rw-r--r-- 1 ps

In [7]:
cat run.debug.sh 

#!/bin/bash -l

#SBATCH -A m1800
#SBATCH -p debug
#SBATCH -N 32
#SBATCH -t 00:30:00
#SBATCH -L SCRATCH  
#SBATCH -J tracer
#SBATCH -C haswell

cd $SLURM_SUBMIT_DIR

srun -n 1024 ../pflotran-cori -pflotranin pflotran_bigplume.in


####SBATCH --qos premium
####SBATCH -p regular
######SBATCH -C haswell  ##need comment out haswell when you're on edison


In [13]:
%sbatch run.debug.sh

'Submitted batch job 13162214\n'

In [21]:
# %mode pandas

ERROR:root:Line magic function `%mode` not found.


In [19]:
%squeue -u pshuai

,JOBID,PARTITION,NAME,USER,ST,TIME,NODES,NODELIST(REASON)


In [18]:
%scancel 13162214

''

## execute interactive job

In [16]:
%salloc -N 1 -q debug -C haswell -t 00:30:00 -L SCRATCH

('',
 'Launched in background. Redirecting stdin to /dev/null\nsalloc: Pending job allocation 12123294\nsalloc: job 12123294 queued and waiting for resources\nsalloc: job 12123294 has been allocated resources\nsalloc: Granted job allocation 12123294\nsrun: error: ioctl(TIOCGWINSZ): Inappropriate ioctl for device\nsrun: error: Not using a pseudo-terminal, disregarding --pty option\nsalloc: Relinquishing job allocation 12123294\n')

In [18]:
%srun -n 32 ../pflotran-cori-flux -pflotranin pflotran.in

('',
 'Launched in background. Redirecting stdin to /dev/null\nsrun: error: No hardware architecture specified (-C)!\nsrun: error: Unable to allocate resources: Unspecified error\n')

In [19]:
%squeue -u pshuai

,JOBID,PARTITION,NAME,USER,ST,TIME,NODES,NODELIST(REASON)


In [27]:
!sh /global/cscratch1/sd/pshuai/test/test.sh

hello world! 


In [30]:
!cat /global/cscratch1/sd/pshuai/test/test.sh

#!/bin/bash -l

 printf "hello world! \n"


